In [ ]:
import numpy as np
from torch.utils.data import Dataset
from torch import from_numpy
import torch
import torchvision
from PIL import Image

In [ ]:
transform=torchvision.transforms.Compose([            
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5),(0.5))])

class dem_dataset(Dataset):
    
    def __init__(self,DF):

        """
        Las entradas de la red neuronal son
        x: coordenada espacial x
        y: coordenada espacial y
        Profundidad: coordenada espacial z
        DEM: path del dem
        Pendite: path de la pendiente    

        encode_class: Clase codificada   
        
        """
        self.x=DF[['x', 'y', 'Profundidad','DEM', 'Pendiente']]
        self.y=from_numpy(np.vstack(DF['encode_class'].values))
        
        self.n_samples=self.x.shape[0]

    
    def __getitem__(self,index):
        
        dem = Image.open(self.x["DEM"].iloc[index])
        dem = transform(dem)
        
        pendiente = Image.open(self.x["Pendiente"].iloc[index])
        pendiente = transform(pendiente)        
        

        corrs = self.x[['x', 'y', 'Profundidad']].iloc[index].values
        
        coordenadas = torch.from_numpy(corrs)

        coordenadas[-1] = coordenadas[-1]*10

        # positional_encode es una instancia de la clase positional_3D_encode
        coordenadas = positional_encode(coordenadas)
        
        target = self.y[index]
        
        return torch.cat([dem.float(),pendiente.float()],0),coordenadas.float(),target
        
    def __len__(self):
        return  self.n_samples